In [1]:
import torch
import logging
from pytorch_pretrained_bert import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from torch import nn
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import string

In [2]:
torch.cuda.is_available()

True

In [3]:
graphics_cards = {
                "Colorful iGame GeForce RTX 3050 Ultra W DUO OC 8G-V 8GB GDDR6 Graphics Card":"11999LE",
                  "GALAX GeForce RTX 3060 8GB 1-Click OC 8GB GDDR6 Graphics Card":"13500LE",
                  "GALAX GeForce RTX 3060 Ti LHR 1-Click OC Feature 8GB GDDR6 Graphics Card":"16900LE",
                  "Colorful GeForce GTX 1660 SUPER NB 6GB V2-V GDDR6 Graphics Card":"8900LE",
                  "COLORFUL IGAME GEFORCE RTX 3060 ULTRA W OC 12G L-V Graphics Card":"15000LE",
                  "ASUS TUF Gaming GeForce RTX 3090 OC 24G":"39500LE",
                    "GIGABAYT GeForce RTX 2060 WINDFORCE OC 12G":"10999LE"
                 }
gc_s = {
                "Colorful iGame GeForce RTX 3050 Ultra W DUO OC 8G-V 8GB GDDR6 Graphics Card":"Colorful iGame RTX 3050 8GB GDDR6",
                  "GALAX GeForce RTX 3060 8GB 1-Click OC 8GB GDDR6 Graphics Card":"GALAX RTX 3060 8GB",
                  "GALAX GeForce RTX 3060 Ti LHR 1-Click OC Feature 8GB GDDR6 Graphics Card":"RTX 3060 Ti",
                  "Colorful GeForce GTX 1660 SUPER NB 6GB V2-V GDDR6 Graphics Card":"GTX 1660 SUPER 6GB",
                  "COLORFUL IGAME GEFORCE RTX 3060 ULTRA W OC 12G L-V Graphics Card":"GEFORCE RTX 3060 ULTRA",
                  "ASUS TUF Gaming GeForce RTX 3090 OC 24G":"RTX 3090 OC 24G",
                    "GIGABAYT GeForce RTX 2060 WINDFORCE OC 12G":"RTX 2060 WINDFORCE OC 12G"
                 }



processors =  {"INTEL CORE I5 10400F TRAY Processor":"4299LE",
               "Intel Core i9-13900K Processor":"23900LE",
               "Intel Core i9 13900 24 Cores 32 Threads Max Turbo 5.6GHz 36MB Cache LGA 1700 Socket 13th Gen Raptor Lake":"22350LE",
               "AMD Ryzen 7 5800X BOX Desktop Processors (8 CORE-16 THREAD) (32M CACHE/3.8GHz/UP TO 4.7GHz)":"9199LE",
               "AMD Ryzen 5 3600 TRAY-6CORE-12THREAD-32M CACH/3.6GHZ up to 4.2GHZ-PCIe 4.0":"3700LE"
               
              }
p_s =  {"INTEL CORE I5 10400F TRAY Processor":"CORE I5 10400F",
               "Intel Core i9-13900K Processor":"Core i9-13900K",
               "Intel Core i9 13900 24 Cores 32 Threads Max Turbo 5.6GHz 36MB Cache LGA 1700 Socket 13th Gen Raptor Lake":"Core i9 13900",
               "AMD Ryzen 7 5800X BOX Desktop Processors (8 CORE-16 THREAD) (32M CACHE/3.8GHz/UP TO 4.7GHz)":"AMD Ryzen 7 5800X",
               "AMD Ryzen 5 3600 TRAY-6CORE-12THREAD-32M CACH/3.6GHZ up to 4.2GHZ-PCIe 4.0":"AMD Ryzen 5 3600"}

ram = {"HyperX Impact 8GB DDR4 3200MHz Memory RAM SODIMM":"850",
       "DATO RAM 32GB DDR4 P2666 NOTEBOOK":"2600",
       "Crucial 8GB DDR4-3200 SODIMM 1.2V CL22 LAPTOP":"730",
       "LEXAR RAM 8G P2666 CL22 SODIMM NOTEBOOK":"650",
       "RAM-TEAM-ELITE-16G-3200HZ-NB-Cl22-22-22":"1600"
    
}

r_s = {"HyperX Impact 8GB DDR4 3200MHz Memory RAM SODIMM":"8GB RAM",
       "DATO RAM 32GB DDR4 P2666 NOTEBOOK":"32GB RAM",
       "Crucial 8GB DDR4-3200 SODIMM 1.2V CL22 LAPTOP":"8GB RAM",
       "LEXAR RAM 8G P2666 CL22 SODIMM NOTEBOOK":"8GB RAM",
       "RAM-TEAM-ELITE-16G-3200HZ-NB-Cl22-22-22":"16GB RAM"
    
}

data = {"intents": [

          {"tag": "greeting",
          "patterns": ["Hi","How are you?","Is anyone there?","Hello","Good day","What's up","Good Morning","how are ya","heyy","whatsup",
      "??? ??? ??"],
          "responses": ["Hi how can I serve you?"],
         },
        {"tag": "pleasure",
          "patterns": [ "It was a pleasure to meet you.","It is a pleasure to meet you.","It was nice to meet you.","Lovely to meet you.","Great to meet you!","Glad to meet you.","Nice meeting you.","Pleasure to meet you.","It was an honor to meet you.","It was a pleasure meeting you.","Looking forward to meeting you again.","It's a pleasure to finally meet you.","It was a pleasure to meet someone with your expertise.","Hi! Nice to meet you.","Hey there!","Howdy!","Well, this is a nice surprise!"],
          "responses": ["thanks it was a pleasure to meet you too"],
         },
          {"tag": "goodbye",
          "patterns": [ "bye", "later","goodbye","see you later", "Goodbye!", "Bye!", "See ya!", "Later!", "Cya!", "Gotta go!", "Talk to you later!", "Adios!", "Ciao!", "Catch you later!", "Take care!", "Peace out!", "TTYL!", "Brb!", "Off now!", "Later gator!","See ya in a bit!", "Deuces!"],
          "responses": ["It was nice to serve you"]
        },
         {"tag": "name",
          "patterns": ["What's your name?", "What should I call you?", "What's up, stranger?", "What's your username?", "Mind if I know your name?", "How can I address you?", "Hey there! What's your name?", "Yo, what's your name?", "Name's me, what's yours?", "What's the name?", "Spill the beans, who are you?", "Let's get acquainted, what's your name?", "Curious, what's your name?","Introduce yourself? ", "What should I call this amazing person?",  "Name please? (so I don't call you dude all the time)"],
          "responses": ["My name is Alphabot, I was created by ALPHA COMPUTER STORE to help our customers tom make thier descisions about shopping for our products"]
         },
         {
          "tag": "website",
          "patterns": ["How can I order?", "How to order online?", "Can I order online?",
            "How can I order something from your website?", "I want to make an order",
            "I want the link of your website", "What products do you have?",
            "How do I order something?", "I want to see your products",
            "Looking to place an order, how do I do that?",  # More explicit
            "Can I buy something online?",  # Simpler way of asking about ordering
            "Do you have an online store?",  # Asks about online ordering possibility
            "Is there a way to purchase your products online?",  # More formal inquiry
            "Need to place an order, what are the options?",  # Asks for ordering details
            "I'm interested in ordering. Can you tell me how?",  # Shows buying intent
            "Let me know how I can buy something from you.",  # Direct request
            "Website question: How can I order?",  # Clarifies website purpose
            "Just browsing, but curious about ordering?",  # Browsing with intent
            "Is there a menu/catalog I can look at to order?",  # Asks for product info
            "Ready to buy! How do I place an order?"],
          "responses": ["You can shop and order our products at our website https://www.alpha-computer.com"]     
        },
        {
          "tag": "About us",
          "patterns": [    "How can I order?", "How to order online?", "Can I order online?",
                        "How can I order something from your website?", "I want to make an order",
                        "I want the link of your website", "What products do you have?",
                        "How do I order something?", "I want to see your products",
                        "Looking to place an order, how do I do that?",  # More explicit
                        "Can I buy something online?",  # Simpler way of asking about ordering
                        "Do you have an online store?",  # Asks about online ordering possibility
                        "Is there a way to purchase your products online?",  # More formal inquiry
                        "Need to place an order, what are the options?",  # Asks for ordering details
                        "I'm interested in ordering. Can you tell me how?",  # Shows buying intent
                        "Let me know how I can buy something from you.",  # Direct request
                        "Website question: How can I order?",  # Clarifies website purpose
                        "Just browsing, but curious about ordering?",  # Browsing with intent
                        "Is there a menu/catalog I can look at to order?",  # Asks for product info
                        "Ready to buy! How do I place an order?"  ],
          "responses": ["We are Alpha store,  We are specializing in retail and wholesale electronics including laptops, computers and accessories. \n\nwe sell : \n 1-Graphics Cards \n 2-Processors \n 3-Ram \n 4-Motherboards \n 5-Power Supplyies \n 6-Storage Cards \n 7-Monitors \n 8-Electronic accessories \n \n and many other products"]     
        },
        {
          "tag": "contact us",
          "patterns": ["What's your phone number?", "Phone number, please?", "Could you share your phone number?", "Can I have your phone number?", "Your phone number, please?", "May I get your phone number?", "Could you provide your phone number?", "What's your contact number?", "Can you give me your phone number?", "Could you please share your phone number?", "What's your cellphone number?", "Your phone number, if you don't mind?", "May I know your phone number?", "Could you tell me your phone number?", "Can you provide your phone number?", "What is your email?", "Could you please share your email?", "What's your email address?", "Can I have your email?", "Your email, please?"],
          "responses": ["You can contact us by \n Phone: 010012345678 \n Email: alpha@store.com \n our help center: https://www.alpha-computer\helpCenter"]     
        },
        {"tag": "thanks",
          "patterns": ["Thank you", "Thankyou", "Thanks you helped me a lot", "Thanks for the help", "Thanks a bunch", "Thanks a ton", "Thanks a million", "Thank you so much", "Thanks a bunch for your assistance", "Appreciate your help", "Thanks for your support", "Much appreciated", "Thank you very much", "Thanks for everything", "Thank you kindly", "Thanks for your time", "Thank you for your kindness", "Thanks a heap", "Thanks for your guidance", "Thanks for being so helpful"],
          "responses": ["I am always happy to help"],
         }
        
        
    
    
             
    

]}






<>:113: SyntaxWarning: invalid escape sequence '\h'
<>:113: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Dell\AppData\Local\Temp\ipykernel_4640\1420559582.py:113: SyntaxWarning: invalid escape sequence '\h'
  "responses": ["You can contact us by \n Phone: 010012345678 \n Email: alpha@store.com \n our help center: https://www.alpha-computer\helpCenter"]


In [4]:
for key in graphics_cards:
    data["intents"].append(
        {
          "tag": f"Graphics cards or GPU : {key}",
          
            "patterns": [f"I want to order {key}",f"I want to make an order for {key}",f"is {key} available",f'I want a {key}'
                       ,f"do you have{key}",f"I want to order {gc_s[key]}",f"I want to make an order for {gc_s[key]}",
                       f"is {gc_s[key]} available",f'I want a {gc_s[key]}',f"do you have{gc_s[key]}"],
          
            "responses": [f"You can find '{key}' in our website https://www.alpha-computer/LINK_FOR_PRODUCT.com"]     
        }
    )
    data["intents"].append(
    {
          "tag": f"Graphics cards or GPU cost : {key}",
          "patterns": [f" how much is {key}",f" how much does {key} cost",f"I want to make an order for {key} how much does it cost",f"is {key} cost alot?",
                      f"I want to make an order for {gc_s[key]}",f"is {gc_s[key]} available",f'I want a {gc_s[key]}',f"do you have{gc_s[key]}"],
          "responses": [f"You can consider '{key}' from our store with only '{graphics_cards[key]}' take a look to it in https://www.alpha-computer/LINK_FOR_PRODUCT.com"]     
        }
    )

for key in processors:
    data["intents"].append(
        {
          "tag": f"Processors or CPU: {key}",
          "patterns": [f"I want to order {key}",f"I want to make an order for {key}",f"is {key} available",
                      f"I want to order {p_s[key]}",f"I want to make an order for {p_s[key]}",f"is {p_s[key]} available",
                       f'I want a {p_s[key]}',f"do you have{p_s[key]}"],
          "responses": [f"You can find '{key}' and many other processors models in our website https://www.alpha-computer/LINK_FOR_PRODUCT.com"]     
        }
    )
    data["intents"].append(
    {
          "tag": f"Processors or CPU cost : {key}",
          "patterns": [f"I want to order {key}",f" how much is {key}",f" how much does {key} cost",
                       f"I want to make an order for {key} how much does it cost",f"is {key} cost alot?",
                       f"which models of {key} do you have",f"I want to make an order for {p_s[key]}",
                       f"is {p_s[key]} available",f'I want a {p_s[key]}',f"do you have{p_s[key]}"],
          "responses": [f"You can consider '{key}' from our store with only '{processors[key]}' take a look at our online shop in https://www.alpha-computer/LINK_FOR_PRODUCT.com"]     
        }
    )


    

gc_list= 'We offer many models of graphics cards this is a list of our top requisted products:\n\n'
for i, key in enumerate(graphics_cards):
    gc_list += f"{i+1}- {key} \n price: {graphics_cards[key]}\n\n"

pc_list= 'We offer many models of Processors this is a list of our top requisted products:\n\n'
for i, key in enumerate(processors):
    pc_list += f"{i+1}- '{key}' \n price: {processors[key]}\n\n"

ram_list= 'We offer many models of Rams this is a list of our top requisted products:\n\n'
for i, key in enumerate(processors):
    pc_list += f"{i+1}- '{key}' \n price: {processors[key]}\n\n"


data["intents"].append(
        {
          "tag": f"Graphics cards list",
          "patterns": [f"What kind of graphics cards do you have?","list me the gpus that you are selling","list me the every gpu you are selling","recommend a Graphics card for me",f"I want a Graphics card","I want gtx or rtx","I want a Gtx graphics card","I want a Rtx graphics card","I am missing a graphics card","which graphics cards do you offer?","suggest a graphics card for me","do you sell graphics card?","do you have graphics card?","do you have good graphics card to sell?","what kind of graphics card do you have?","suggest a graphics card for me",
                      f"What kind of GPU do you have?","list me the Graphics cards that you are selling","list me the every Graphics card you are selling","recommend a GPU for me",f"I want a GPU","I want gtx or rtx","I want a Gtx GPU","I want a Rtx GPU","I am missing a GPU","which GPU do you offer?","suggest a GPU for me","do you sell GPU?","do you have GPU?","do you have good GPU to sell?","what kind of GPU do you have?","suggest a GPU for me","I also need a gpu"],
          "responses": [gc_list]     
        }
    )
data["intents"].append(
        {
          "tag": f"Processor list",
          "patterns": [f"What kind of Processors do you have?","list me the processors that you are selling","recommend a Processor for me",f"I want a Processor","I want Intel or AMD","I am missing a processor","which model of processors do you offer?","do you sell processors?","do you have processors?","do you have good processors to sell?","what kind of processors do you have?","suggest a processor for me","I also need a processor",
                      f"What kind of CPUs do you have?","list me the CPUs that you are selling","recommend a CPU for me",f"I want a CPU","I want Intel or AMD","I am missing a CPU","which model of CPU do you offer?","do you sell CPU?","do you have CPUs?","do you have good CPUs to sell?","what kind of CPUs do you have?","suggest a CPU for me","I also need a CPU"],
          "responses": [pc_list]     
        }
    )

In [5]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
MAX_length = 0
input_sizes = [32,64,128,256,512,1024]
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        MAX_length = max([MAX_length, len(pattern)])
        
        documentX.append("[CLS] " + pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])


for i in range(len(input_sizes)):
    if MAX_length <  input_sizes[i]:
        MAX_length = input_sizes[i]
        break


ourClasses = sorted(set(ourClasses))# sorting classes

In [6]:
print(MAX_length)

256


In [7]:
from sklearn.preprocessing import LabelEncoder

# Define a custom dataset class to handle data loading
class CustomDataset(Dataset):
    def __init__(self, X, Y, tokenizer, max_length):
        self.X = X
        self.Y = Y
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_encoder = LabelEncoder()
        self.Y_encoded = self.label_encoder.fit_transform(Y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        text = self.X[idx]
   
        label = torch.tensor(self.Y_encoded[idx])

        
        # Tokenize input text
        encoding = self.tokenizer(text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt')

        input_ids = encoding['input_ids']
       
        attention_mask = encoding['attention_mask']

        return input_ids, attention_mask, label

In [8]:
len(ourClasses)

34

In [9]:
from transformers import AutoTokenizer, BertForSequenceClassification , AutoModelForSequenceClassification
from transformers import AdamW



tokenizer = AutoTokenizer.from_pretrained("shahrukhx01/question-vs-statement-classifier")
model = AutoModelForSequenceClassification.from_pretrained("shahrukhx01/question-vs-statement-classifier", 
                                                           num_labels=len(ourClasses),ignore_mismatched_sizes=True
                                                         )

model.train()


# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


C:\Users\Dell\Documents\GitHub\mostafa_gp\chatbot\torchenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at shahrukhx01/question-vs-statement-classifier and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([34]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 256]) in the checkpoint and torch.Size([34, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 256, padding_idx=0)
      (position_embeddings): Embedding(512, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, e

In [10]:

batch_size = 16
epochs= 100
# Define train loader
train_dataset = CustomDataset(documentX, documentY, tokenizer, max_length= MAX_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [11]:


# Define Loss Function (replace with your desired loss function)
criterion = nn.CrossEntropyLoss()

# Define Optimizer

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

# Training Loop
for epoch in range(epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    for inputs, attention_mask , labels in train_loader:
       
        
        # Move inputs and labels to device
        labels = labels.to(device)
     
        inputs = inputs.view(inputs.shape[0], MAX_length).to(device)
   
        attention_mask = attention_mask.to(device)
       
        # Forward pass
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)

        # Get predictions
        _, predicted = torch.max(outputs.logits, 1)

        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        
        # Calculate loss
        loss = criterion(outputs.logits, labels)
        total_loss += loss.item() * inputs.size(0)

        # Backward pass and optimize
        optimizer.zero_grad()
        #loss.backward()
        loss.backward()
        optimizer.step()
    # Calculate accuracy
    print("predicted",predicted)
    print("True ",labels)
    accuracy = correct_predictions / total_samples
    # ... (Track and print training progress)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_dataset):.4f}, Accuracy: {accuracy:.4f}')


C:\Users\Dell\Documents\GitHub\mostafa_gp\chatbot\torchenv\Lib\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


predicted tensor([20, 20, 27, 20, 20], device='cuda:0')
True  tensor([ 8, 20, 29, 28,  1], device='cuda:0')
Epoch [1/100], Loss: 3.5447, Accuracy: 0.0356
predicted tensor([30,  5, 15, 28, 17], device='cuda:0')
True  tensor([31,  8,  4, 23,  9], device='cuda:0')
Epoch [2/100], Loss: 3.5049, Accuracy: 0.0546
predicted tensor([33, 20, 27,  4, 33], device='cuda:0')
True  tensor([27,  5, 10, 21, 20], device='cuda:0')
Epoch [3/100], Loss: 3.4776, Accuracy: 0.0736
predicted tensor([28, 27, 28, 28, 28], device='cuda:0')
True  tensor([ 1, 27, 25, 32,  1], device='cuda:0')
Epoch [4/100], Loss: 3.4339, Accuracy: 0.1045
predicted tensor([33, 28, 20, 28, 27], device='cuda:0')
True  tensor([29, 26,  9, 33, 12], device='cuda:0')
Epoch [5/100], Loss: 3.4110, Accuracy: 0.1093
predicted tensor([27, 28, 28, 17, 26], device='cuda:0')
True  tensor([ 9, 28, 24, 14,  6], device='cuda:0')
Epoch [6/100], Loss: 3.3826, Accuracy: 0.1235
predicted tensor([ 4, 27, 33, 28, 27], device='cuda:0')
True  tensor([ 3, 33

In [12]:
model.save_pretrained('question_vs_statement_classifier')


In [13]:
import time

def getRes(message):
    encoding = tokenizer(message,
            add_special_tokens=True,
            max_length=MAX_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt')

    input_ids = encoding['input_ids'].to(device)
       
    attention_mask = encoding['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    predicted = torch.argmax(output.logits ,1)
    tag = train_dataset.label_encoder.inverse_transform(predicted.cpu())
    for intent in data["intents"]:

        if intent["tag"] == tag[0]:
            print(intent["responses"])
            break
       
 


while True:
    newMessage = input("")
    ourResult = getRes(newMessage)
    print('\n')
    

 greetings


['It was nice to serve you']




 what do you sell?


['My name is Alphabot, I was created by ALPHA COMPUTER STORE to help our customers tom make thier descisions about shopping for our products']




 How can I order?


['You can shop and order our products at our website https://www.alpha-computer.com']




 do you sell ram ?


['We offer many models of graphics cards this is a list of our top requisted products:\n\n1- Colorful iGame GeForce RTX 3050 Ultra W DUO OC 8G-V 8GB GDDR6 Graphics Card \n price: 11999LE\n\n2- GALAX GeForce RTX 3060 8GB 1-Click OC 8GB GDDR6 Graphics Card \n price: 13500LE\n\n3- GALAX GeForce RTX 3060 Ti LHR 1-Click OC Feature 8GB GDDR6 Graphics Card \n price: 16900LE\n\n4- Colorful GeForce GTX 1660 SUPER NB 6GB V2-V GDDR6 Graphics Card \n price: 8900LE\n\n5- COLORFUL IGAME GEFORCE RTX 3060 ULTRA W OC 12G L-V Graphics Card \n price: 15000LE\n\n6- ASUS TUF Gaming GeForce RTX 3090 OC 24G \n price: 39500LE\n\n7- GIGABAYT GeForce RTX 2060 WINDFORCE OC 12G \n price: 10999LE\n\n']




KeyboardInterrupt: Interrupted by user

In [29]:
_, predicted = torch.max(torch.tensor([10,453,654,8,6,3]), 1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)